# Deploy Boltz2 NIM from AWS Marketplace

The Boltz-2 NIM provides state-of-the-art biomolecular structure prediction and binding affinity prediction capabilities for combinations of proteins, RNA, DNA, and other molecules. Based on the Boltz-2 architecture, this NIM enables researchers to predict complex biomolecular structures with high accuracy and quantify molecular interactions, supporting a wide range of molecular configurations and binding studies.

Boltz-2 represents a significant advancement in computational biology, offering unprecedented capabilities for predicting:

- **Protein structures**: Single and multi-chain protein complexes

- **Nucleic acid structures**: DNA and RNA molecules in various configurations

- **Protein-nucleic acid complexes**: Interactions between proteins and genetic material

- **Ligand binding and affinity prediction**: Small molecule interactions with biomolecules, including predicted binding affinity scores

- **Modified residues**: Post-translational modifications and chemical modifications

- **Constraint-guided predictions**: Structure predictions conditioned on specified interaction pockets or contacts

The model supports both single-molecule predictions and complex multi-molecular assemblies, making it suitable for a wide range of research applications from basic structural biology to drug discovery.


For detailed technical information about the Boltz architecture and training methodology, refer to the Boltz-1 technical report: [Wohlwend et al. 2024.](https://doi.org/10.1101/2024.11.19.624167)

Please check out the [Boltz2 NIM docs](https://docs.nvidia.com/nim/bionemo/boltz2/latest/overview.html) and [NVIDIA AI Build Link](https://build.nvidia.com/) for more information.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to one of the models listed above.


## Subscribe to the model package
To subscribe to the model package:
1. Open the model package listing page
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
import boto3, json, sagemaker, time, os
from sagemaker import get_execution_role, ModelPackage
from botocore.config import Config

config = Config(read_timeout=10000)
sess = boto3.Session()
sm = sess.client("sagemaker")
sagemaker_session = sagemaker.Session(boto_session=sess)
role = get_execution_role()
client = boto3.client("sagemaker-runtime", config=config)
region = sess.region_name

In [ ]:
# Verify/replace the arn below with the model package arn you want to deploy
nim_package = "boltz2-nim-v1-4-0-8bf338ab871f3838b142bd8a5d294deb"

# Mapping for Model Packages
model_package_map = {
    "us-east-1": f"arn:aws:sagemaker:us-east-1:865070037744:model-package/{nim_package}",
    "us-east-2": f"arn:aws:sagemaker:us-east-2:057799348421:model-package/{nim_package}",
    "us-west-1": f"arn:aws:sagemaker:us-west-1:382657785993:model-package/{nim_package}",
    "us-west-2": f"arn:aws:sagemaker:us-west-2:594846645681:model-package/{nim_package}",
    "ca-central-1": f"arn:aws:sagemaker:ca-central-1:470592106596:model-package/{nim_package}",
    "eu-central-1": f"arn:aws:sagemaker:eu-central-1:446921602837:model-package/{nim_package}",
    "eu-west-1": f"arn:aws:sagemaker:eu-west-1:985815980388:model-package/{nim_package}",
    "eu-west-2": f"arn:aws:sagemaker:eu-west-2:856760150666:model-package/{nim_package}",
    "eu-west-3": f"arn:aws:sagemaker:eu-west-3:843114510376:model-package/{nim_package}",
    "eu-north-1": f"arn:aws:sagemaker:eu-north-1:136758871317:model-package/{nim_package}",
    "ap-southeast-1": f"arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/{nim_package}",
    "ap-southeast-2": f"arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/{nim_package}",
    "ap-northeast-2": f"arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/{nim_package}",
    "ap-northeast-1": f"arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/{nim_package}",
    "ap-south-1": f"arn:aws:sagemaker:ap-south-1:077584701553:model-package/{nim_package}",
    "sa-east-1": f"arn:aws:sagemaker:sa-east-1:270155090741:model-package/{nim_package}",
}

region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

model_package_arn = model_package_map[region]
model_package_arn

## Create the SageMaker Endpoint

We first define SageMaker model using the specified ModelPackageArn.

In [ ]:
# Define the model details
sm_model_name = "Boltz2-NIM-v140-MP"

# Create the SageMaker model
create_model_response = sm.create_model(
    ModelName=sm_model_name,
    PrimaryContainer={
        'ModelPackageName': model_package_arn
    },
    ExecutionRoleArn=role,
    EnableNetworkIsolation=True
)
print("Model Arn: " + create_model_response["ModelArn"])

Next we create endpoint configuration specifying instance type

In [ ]:
# Create the endpoint configuration
endpoint_config_name = sm_model_name

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTraffic',
            'ModelName': sm_model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.g6e.24xlarge', 
            'InferenceAmiVersion': "al2-ami-sagemaker-inference-gpu-3-1",
            'RoutingConfig': {'RoutingStrategy': 'LEAST_OUTSTANDING_REQUESTS'},
            'ModelDataDownloadTimeoutInSeconds': 3600, # Specify the model download timeout in seconds.
            'ContainerStartupHealthCheckTimeoutInSeconds': 3600, # Specify the health checkup timeout in seconds
        }
    ]
)
print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

Using the above endpoint configuration we create a new sagemaker endpoint and wait for the deployment to finish. The status will change to InService once the deployment is successful.

In [ ]:
# Create the endpoint
endpoint_name = endpoint_config_name
create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

In [ ]:
resp = sm.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

## Run Inference

### Boltz2 NIM Inference Example-1

Once we have the model deployed we can use a sample payload to do an inference request. For inference request format, currently NIM on SageMaker supports the OpenAI API inference protocol. For explanation of supported parameters please see [Boltz2 Inferencing link](https://docs.nvidia.com/nim/bionemo/boltz2/latest/inference.html).

In [ ]:
runtime = boto3.client('sagemaker-runtime')

# Simple protein structure prediction
payload = {
    "polymers": [
        {
            "id": "A",
            "molecule_type": "protein",
            "sequence": "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"
        }
    ],
    "recycling_steps": 2,
    "sampling_steps": 20,
    "diffusion_samples": 1
}

print("Running protein structure prediction...")
start = time.time()

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

result = json.loads(response['Body'].read())
elapsed = time.time() - start

print(f"✅ Inference completed in {elapsed:.2f}s")
print(f"Structures: {len(result.get('structures', []))}")
print(f"Confidence: {result.get('confidence_scores', [])}")


### Protein-Ligand Complex Prediction


In [ ]:
# Protein + ligand (Aspirin)
payload = {
    "polymers": [
        {
            "id": "A",
            "molecule_type": "protein",
            "sequence": "MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKT"
        }
    ],
    "ligands": [
        {"id": "L1", "smiles": "CC(=O)OC1=CC=CC=C1C(=O)O"}  # Aspirin
    ],
    "recycling_steps": 2,
    "sampling_steps": 20
}

print("Running protein-ligand prediction...")
start = time.time()

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

result = json.loads(response['Body'].read())
elapsed = time.time() - start

print(f"✅ Protein-ligand prediction completed in {elapsed:.2f}s")
print(f"Confidence: {result.get('confidence_scores', [])}")


### Protein-Ligand Complex Prediction with Affinities Predictions


In [ ]:
# Test protein-ligand with affinity prediction
payload_affinity = '''{
  "polymers": [
    {
      "molecule_type": "protein",
      "entity_id": 1,
      "sequence": "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"
    }
  ],
  "ligands": [
    {
      "entity_id": 2,
      "smiles": "CC(=O)Oc1ccccc1C(=O)O",
      "predict_affinity": true
    }
  ]
}
'''

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=payload_affinity
)

result = json.loads(response["Body"].read())
print("Confidence:", result.get("confidence_scores", []))
print("Affinities:", result.get("affinities", {}))

## Cleanup


In [ ]:
# Uncomment to delete resources

# sm.delete_endpoint(EndpointName=endpoint_name)
# sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
# sm.delete_model(ModelName=endpoint_name)
# print("Resources deleted")
